Define variables and import required function.

In [29]:
from math import log
#Define the call outcomes for t<25 and t>25
CALL_OUTCOMES = {("LINE BUSY", 0.2, 0.2, 6+3+1), ("UNAVAILABLE", 0.3, 0.5, 6+25+1), ("AVAILABLE", 0.5, 1.0, 6+1)}
PRECISION = 4

Create a random number generator using linear congruential random number algorithm.

In [30]:
def random_number_gen(x0=1000, a=24693, c=3517, k=(2**17)):
    while True:
        x_i = (a * x0 + c) % k
        yield (x_i/k)
        x0 = x_i

Create random variable generators for both the call outcome $C$ and total call time $X$ to simulate call.

In [31]:
def determine_call_time(random_num): #previous cont_rand_var_gen
    return -12*log(1-random_num)

def simulate_call(random_num): #previously discrete_rand_var_gen(random_num)
    #determine call outcome by iterating through all outcomes and finding first match to CDF
    for outcome in CALL_OUTCOMES:
        if outcome[2] >= random_num:
            call_outcome = outcome
            #if customer is available calculate availability time
            if outcome[0] == "AVAILABLE":
                time_waited = determine_call_time((random_num-0.5)/0.5)
                if time_waited <= 25:
                    call_outcome = ("AVAILABLE", 0.5, 1.0, round(6+time_waited+1, PRECISION))
                else: 
                    call_outcome = ("UNAVAILABLE (X>25)", 0.5, 1.0, 32)
            return call_outcome

Run the simulation.

In [32]:
total_customers_called = 0
total_time_spent = 0
call_time_t = []
total_time_w = []
generator = random_number_gen()

for call_index in range (1000):
    rand_num = next(generator)
    

U_51 = 0.5157089233398438
U_52 = 0.427276611328125
U_53 = 0.7681961059570312
(None, 1.0, None)
('AVAILABLE', 0.5, 1.0, 13.1299)
